In [ ]:
# 모듈화 시켜서 배포
# aws.py
# └ class Rekognition
#   └ compare_faces
#   └ detect_faces
#   └ detect_label
#   └ recognize_celeb

In [10]:
import boto3

def compare_faces(sourceFile, targetFile):

    client = boto3.client('rekognition')

    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')

    response = client.compare_faces(SimilarityThreshold=0,
                                    SourceImage={'Bytes': imageSource.read()},
                                    TargetImage={'Bytes': imageTarget.read()})

    result = ""
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']

        result = f"동일 인물일 확률 :{faceMatch['Similarity'] : .2f}%"
    imageSource.close()
    imageTarget.close()
    if len(response['FaceMatches']) == 0:
        return "얼굴을 인식할 수 없습니다"
    else:
        return result

def main():
    source_file = 'imgs/choi2.jpg'
    target_file = 'imgs/misun.jpeg'
    face_matches = compare_faces(source_file, target_file)
    print(face_matches)
if __name__ == "__main__":
    main()

동일 인물일 확률 : 0.33%


In [19]:
import boto3
import json

def detect_faces(photo):
    
    client = boto3.client('rekognition')

    with open(photo, "rb") as image:
        response = client.detect_faces(Image={'Bytes' : image.read()}, Attributes=['ALL'])

    result = ""
    for faceDetail in response['FaceDetails']:

        # 성별 : 남성   (혹은 여성)
        gender = faceDetail['Gender']['Value']
        result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
        # 나이 : 28(세) (24 ~ 32의 평균)
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
        result += f"나이 : {age}(세)\n"
        # 감정 : HAPPY
        emotion = faceDetail['Emotions'][0]['Type']
        result += f"감정 : {emotion}"

    if len(response['FaceDetails']) == 0:
        return "얼굴을 인식할 수 없습니다"
    else:
        return result
    
def main():
    photo='imgs/choi2.jpg'
    face_count=detect_faces(photo)
    print(face_count)

if __name__ == "__main__":
    main()

성별 : 여성
나이 : 15(세)
감정 : CALM


In [29]:
import boto3

def recognize_celebrities(photo):
    
    client = boto3.client('rekognition')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    result = ""
    # 닮은 연예인이 없는 경우...누가 들어도 괜찮은 사람을 닮았다고 리턴
    celeb_list = ["송중기", "아라아나 그란데", "아이유", "장원영", "카리나", "설윤", "송강", "강동원"]
    
    for celebrity in response['CelebrityFaces']:
        result = celebrity['Name']

    import random
    
    if len(response['CelebrityFaces']) == 0:
        return random.choice(celeb_list)
    else:
        return result

def main():
    photo = 'imgs/무2.jpg'
    celeb_count = recognize_celebrities(photo)
    print("Celebrities detected: " + str(celeb_count))

if __name__ == "__main__":
    main()

Celebrities detected: Jasmila Žbanić


In [51]:
import boto3

# 1) 이미지를 던졌을 때, 해당 label을 return
# 2) 이 이미지가 강아지인지 아닌지 확인하고 싶어서
#    -> "강아지일 확률은 95%입니다"
#    -> 혹은 "강아지가 아닙니다"
def detect_labels_local_file(photo):

    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    result = ""
    for label in response['Labels']:
        result += f"{label['Name']} : {label['Confidence']:.2f}%\n"

    if len(response['Labels']) == 0:
        return "잘못된 이미지입니다"
    else:
        return result

def main():
    photo='imgs/black.jpeg'

    label_count=detect_labels_local_file(photo)
    print(label_count)


if __name__ == "__main__":
    main()

Animal : 100.00%
Canine : 100.00%
Dog : 100.00%
Labrador Retriever : 100.00%
Mammal : 100.00%
Pet : 100.00%
Kangaroo : 88.44%



In [54]:
import boto3

def detect_is_label(photo, key):

    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    result = ""

    is_checked = False
    
    for label in response['Labels']:
        if label['Name'] == key:
            result += f"{key}일 확률은 {label['Confidence']:.2f}%입니다\n"
            is_checked = True
            break

    if len(response['Labels']) == 0:
        return "잘못된 이미지입니다"
    elif is_checked == True:
        return result
    else:
        return f"{key}이(가) 아닙니다"

def main():
    photo='imgs/cat.jpeg'

    key = "Dog"
    label_count=detect_is_label(photo, key)
    print(label_count)

if __name__ == "__main__":
    main()

Dog이(가) 아닙니다


In [38]:
class Rekognition:
    def compare_faces(self, sourceFile, targetFile):

        client = boto3.client('rekognition')
    
        imageSource = open(sourceFile, 'rb')
        imageTarget = open(targetFile, 'rb')
    
        response = client.compare_faces(SimilarityThreshold=0,
                                        SourceImage={'Bytes': imageSource.read()},
                                        TargetImage={'Bytes': imageTarget.read()})
    
        result = ""
        for faceMatch in response['FaceMatches']:
            position = faceMatch['Face']['BoundingBox']
    
            result = f"동일 인물일 확률 :{faceMatch['Similarity'] : .2f}%"
        imageSource.close()
        imageTarget.close()
        if len(response['FaceMatches']) == 0:
            return "얼굴을 인식할 수 없습니다"
        else:
            return result

    def detect_faces(self, photo):
        
        client = boto3.client('rekognition')
    
        with open(photo, "rb") as image:
            response = client.detect_faces(Image={'Bytes' : image.read()}, Attributes=['ALL'])
    
        result = ""
        for faceDetail in response['FaceDetails']:
    
            # 성별 : 남성   (혹은 여성)
            gender = faceDetail['Gender']['Value']
            result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
            # 나이 : 28(세) (24 ~ 32의 평균)
            age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
            result += f"나이 : {age}(세)\n"
            # 감정 : HAPPY
            emotion = faceDetail['Emotions'][0]['Type']
            result += f"감정 : {emotion}"
    
        if len(response['FaceDetails']) == 0:
            return "얼굴을 인식할 수 없습니다"
        else:
            return result
    
    def recognize_celebrities(self, photo):
        
        client = boto3.client('rekognition')
    
        with open(photo, 'rb') as image:
            response = client.recognize_celebrities(Image={'Bytes': image.read()})
    
        result = ""
        # 닮은 연예인이 없는 경우...누가 들어도 괜찮은 사람을 닮았다고 리턴
        celeb_list = ["송중기", "아라아나 그란데", "아이유", "장원영", "카리나", "설윤", "송강", "강동원"]
        
        for celebrity in response['CelebrityFaces']:
            result = celebrity['Name']
    
        import random
        
        if len(response['CelebrityFaces']) == 0:
            return random.choice(celeb_list)
        else:
            return result
    
    def detect_labels_local_file(self, photo):
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
        result = ""
        for label in response['Labels']:
            result += f"{label['Name']} : {label['Confidence']:.2f}%\n"
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다"
        else:
            return result
    
    def detect_is_label(self, photo, key):
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
        result = ""
    
        is_checked = False
        
        for label in response['Labels']:
            if label['Name'] == key:
                result += f"{key}일 확률은 {label['Confidence']:.2f}%입니다\n"
                is_checked = True
                break
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다"
        elif is_checked == True:
            return result
        else:
            return f"{key}이(가) 아닙니다"

In [55]:
from aws import detect_is_label
result = detect_is_label("imgs/dog.jpg", "Cat")
print(result)

ImportError: cannot import name 'detect_is_label' from 'aws' (C:\Users\KCCISTC\Desktop\workspace\aws.py)

In [44]:
!pip install flask

  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)

   -------------------------- ------------- 2/3 [flask]
   ---------------------------------------- 3/3 [flask]



In [ ]:
# 웹 프레임워크 : Flask, Django
# 1) Flask : 마이크로 웹 프레임워크
from flask import Flask # 서버 라우팅, 구동
app = Flask(__name__)
@app.route("/")
def index():
    return "<h1>야호</h1>"
if __name__ == "__main__":
    app.run(host="0.0.0.0")



    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.10.141.3:5000
Press CTRL+C to quit
10.10.141.249 - - [17/Sep/2025 10:57:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:50:20] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:50:37] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:50:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:50:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:51:20] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:51:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:52:51] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:52:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:53:25] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:53:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:54:25] "GET / HTTP/1.1" 200 -
10.10.141.3 - - [17/Sep/2025 11:54:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 11:55:22] "GET / HTTP/1.1" 200 -
10.10.141.3